# Skillfactory - Практический Machine Learning
## 19/02/2018 - Аномалии, работа с признаками, пайплайны (HW)

<center> Шестаков Андрей </center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

# Создание пайплайна и генерация признаков
<center>Шестаков Андрей</center>

В этом задании мы рассмотрим данные с предыдущего Sberbank Data Science Contest. К сожалению найти страницу с конкурсом уже не получается.

Одной из задач была опредление пола владельца карты по его транзакциям на карте. Зачем это нужно - одному сберу известно, но эта задача была хороша тем, что в ней можно нагенерировать много разных признаков

Есть такая [презентация](https://alexanderdyakonov.files.wordpress.com/2016/10/dj2016_sdsj_vis.pdf) с предварительным анализом данных и идеями про признаки

Нам понадобятся файлы `customers_gender_train.csv`, `transactions.tsv.gz`, `mcc_types.tsv` и `trans_types.tsv`.

## Посмотрим на данные

Это метки ответов

In [2]:
df_gender = pd.read_csv('data/customers_gender_train.csv')
df_gender.head()

,customer_id,gender
0,75562265,0
1,10928546,1
2,69348468,1
3,84816985,1
4,61009479,0


Это сами транзакции (отрицательные транзакции - списывание, положительные - зачисление на счет)

In [3]:
df_transactions = pd.read_csv('data/transactions.csv.gz')
df_transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


Далее, расшифровки кодов [mcc](https://ru.wikipedia.org/wiki/Merchant_Category_Code) и транзакций

In [4]:
df_tr = pd.read_csv('data/tr_types.csv', sep=';')
df_tr.head()

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...


In [5]:
df_mcc = pd.read_csv('data/tr_mcc_codes.csv', sep=';')
df_mcc.head()

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


Первое что мы видем - это странная дата и суммы в транзакциях. 

В принципе, посмотрев на исходное распределение "относительных" дат по какой-нибудь гендерной группы mcc, становится примерно понятно, что за даты закодированы.

Ну а суммы транзакций организаторы просто умножили на $\pi^{\exp}$ =)

Преобразование будет проделано ниже, но при желании, можете сами со всем разобраться.

## Генерим признаки
В качестве базовых признаков, можно взять, например, 
* количество (доля) транзакций по каждому mcc_code
* количество (доля) транзакций в разные промежутки времени

In [6]:
from pandas import Timestamp, DateOffset

In [7]:
import math

In [8]:
# приведем траты к нормальным значениям - поедлим на Пи**е
df_transactions.loc[df_transactions['amount'].isnull(), 'amount'] = df_transactions['amount'].median()
df_transactions['amount_true'] = df_transactions['amount'] / (math.pi**math.e)

In [9]:
# присвоим каждому id общую сумму, которую он потратил. хорошо бы среднемесячную или что-то в этом роде, но пока пусть так
df_transactions_group = df_transactions.loc[:,['customer_id','amount_true']].groupby(by='customer_id').sum().reset_index()
df_transactions_group.rename(index=str, columns={"amount_true": "total"}, inplace=True)

In [10]:
# посмотрим что получилось 
df_transactions_group.describe()

,customer_id,total
count,1.500000e+04,1.500000e+04
mean,5.036616e+07,-4.176621e+05
std,2.877617e+07,1.474580e+06
min,6.815000e+03,-6.704297e+07
25%,2.546323e+07,-4.580288e+05
50%,5.046361e+07,-2.162148e+05
75%,7.506343e+07,-4.946331e+04
max,9.999968e+07,2.800571e+07


In [11]:
# зададим минимум и максимум потраченных значений
total_min = int(df_transactions_group['total'].min())
total_max = int(df_transactions_group['total'].max())
total_step = int((total_max-total_min)/20)

i = []
for x in range (total_min, total_max, total_step):
    i.append(x)
# раскидаем по катгориям    
df_transactions_group['total_cat'] = df_transactions_group['total'].apply(lambda x:  '0' if x < i[0] else 
                                     '1' if x < i[1] else 
                                     '2' if x < i[2] else 
                                     '3' if x < i[3] else 
                                     '4' if x < i[4] else
                                     '5' if x < i[5] else
                                     '6' if x < i[6] else
                                     '7' if x < i[7] else
                                     '8' if x < i[8] else
                                     '9' if x < i[9] else
                                     '10' if x < i[10] else
                                     '11' if x < i[11] else
                                     '12' if x < i[12] else
                                     '13' if x < i[13] else
                                     '14' if x < i[14] else
                                     '15' if x < i[15] else
                                     '16' if x < i[16] else
                                     '17' if x < i[17] else
                                     '18' if x < i[18] else
                                     '19' if x < i[18] else
                                     '20' if x < i[20] else
                                     '21'
                                    )

In [12]:
"""df_transactions_amount = df_transactions.loc[:,['customer_id','amount']].groupby(by='customer_id').sum()\
                                        .merge(df_gender, on = 'customer_id', how = 'left')                   """

"df_transactions_amount = df_transactions.loc[:,['customer_id','amount']].groupby(by='customer_id').sum()                                        .merge(df_gender, on = 'customer_id', how = 'left')                   "

In [13]:
del df_transactions_group['total']

In [14]:
# обобщим категрии транзакций
df_tr['tr_category'] = df_tr['tr_type'].apply(lambda x:  'Покупка' if x < 2000 else 
                                     'Выдача наличных' if x < 2320 else 
                                     'Платеж скарты/безналичный перевод'   if x < 2325 else 
                                     'Перевод с карты на карту через POS/ATM' if x < 2350 else 
                                     'Погашение кредита' if x < 2370 else
                                     'перевода с карты на карту> через Мобильный банк' if x < 2400 else
                                     'Перевод с карты на счет' if x < 2900 else
                                     'Выдача наличных средств без предъявления карты' if x < 2990 else
                                     'Списание средств' if x < 3001 else
                                     'Плата/коммиччия за обслуживаниt/предоставление' if x < 4000 else
                                     'Плата за получение наличны' if x < 4030 else
                                     'Плата за взнос наличных' if x < 4050 else
                                     'Плата за перевод на карту' if x < 4080 else
                                     'Оплата услуг банка' if x < 4100 else
                                     'Плата за получение наличных' if x < 5110 else
                                     'Списание % за овердрафт' if x < 6000 else
                                     'Возврат  покупки.' if x < 7000 else
                                     'Взнос наличных' if x < 7030 else
                                     'Перевод на карту (с карты)' if x < 7050 else
                                     'Пополнение счета для погашения задолженности' if x < 7070 else
                                     'Перевод с карты на карту в овердрафте / пополнение со счета на карту' if x < 7090 else
                                     'Межфилиальные пополнения зарплатных карт ' if x < 8000 else
                                     'Списание/ перевод / корректировка  ит .д.' if x < 8240 else
                                     'XXX' if x == 999999  else 
                                     'Другое'
                                    )
df_tr['tr_num'] = df_tr['tr_type'].apply(lambda x:  '0' if x < 2000 else 
                                     '1' if x < 2320 else 
                                     '2'   if x < 2325 else 
                                     '3' if x < 2350 else 
                                     '4' if x < 2370 else
                                     '5' if x < 2400 else
                                     '6' if x < 2900 else
                                     '7' if x < 2990 else
                                     '8' if x < 3001 else
                                     '9' if x < 4000 else
                                     '10' if x < 4030 else
                                     '11' if x < 4050 else
                                     '12' if x < 4080 else
                                     '13' if x < 4100 else
                                     '14' if x < 5110 else
                                     '15' if x < 6000 else
                                     '16' if x < 7000 else
                                     '17' if x < 7030 else
                                     '18' if x < 7050 else
                                     '19' if x < 7070 else
                                     '20' if x < 7090 else
                                     '21' if x < 8000 else
                                     '22' if x < 8240 else
                                     '23' if x == 999999  else 
                                     '24'
                                    )

In [15]:
def preproc_transactions(df_transactions):
    sec_per_day = 86400
    sec_per_hour = 3600
    
    start_date = 1420070400 - 154 * sec_per_day - 3 * sec_per_hour
    
    df_transactions.loc[:, 'day'] = df_transactions.tr_datetime\
                                               .str.split(' ')\
                                               .str.get(0)\
                                               .astype(int)
    df_transactions.loc[:, 'time_raw'] = df_transactions.tr_datetime\
                                                    .str.split(' ')\
                                                    .str.get(1)

    # set temp dt
    df_transactions.loc[:, 'dt_temp'] = pd.to_datetime(df_transactions.loc[:, 'time_raw'], 
                                                    format='%H:%M:%S')\
                                        + DateOffset(years=115)
    
    df_transactions = df_transactions.assign(dt = lambda x: x.dt_temp.astype(np.int64) // 10**9
                                             + (x.day - 153) * sec_per_day)\
                                     .assign(weekday = lambda x: ((x.day + 4) % 7 + 1))
        
    df_transactions.loc[:, 'datetime'] = pd.to_datetime(df_transactions.dt, unit='s')
    df_transactions.loc[:, 'date'] = df_transactions.loc[:, 'datetime'].dt.strftime('%Y-%m-%d')
    df_transactions.loc[:, 'hour'] = df_transactions.loc[:, 'datetime'].dt.strftime('%H')
    
    df_transactions = df_transactions.drop(['dt_temp', 'time_raw', 'tr_datetime'], axis=1)
    
    df_transactions.loc[:, 'amount'] = np.round(df_transactions.loc[:, 'amount']/(np.pi**np.exp(1)))
            
    return df_transactions

In [16]:
df_transactions = df_transactions.pipe(preproc_transactions)

In [17]:
#добавим описания типов транзацкицй
df_transactions_merged = df_transactions.merge(df_tr, how = 'left', on = 'tr_type')

In [18]:
# сводная таблица со столбцами - типами транзакций и значениями - количествами транзацкий
df_tr_type_counts = \
        df_transactions_merged.pivot_table(index=['customer_id'], columns='tr_num', values='amount', 
                          aggfunc=np.size, fill_value=0)                            

In [19]:
def gen_features(df_gender, df_transactions, df_transactions_merged\
                 #, df_transactions_group
                ):  

# Сводная таблица №1. "customer_id и mcc коды. в ячейках - количество операций"
    df_mcc_counts = \
    df_transactions.pivot_table(index=['customer_id'], columns='mcc_code', values='amount', 
                             aggfunc=np.size, fill_value=0) # тож что и  aggfunc='count'
    
    # переименование просто столбцов с номерами по заданному форомату: "mcc_{}_count"    
    df_mcc_counts = df_mcc_counts.rename_axis(lambda x: 'mcc_{}_count'.format(x), axis=1)

    
# Сводная таблица №2 "количество операций каждого клиента в каждом часе"    
    df_hour_rations = \
        df_transactions.pivot_table(index=['customer_id'], columns='hour', values='amount', 
                             aggfunc=np.size, fill_value=0)            
    
    # Сложная и нетривиальная конструкция - раскидывает часы по категориям
    total = df_hour_rations.sum(axis=1)
    df_hour_rations.loc[:, 'morning'] = (df_hour_rations.loc[:, '05':'11'].sum(axis=1).T/total).T
    df_hour_rations.loc[:, 'day'] = (df_hour_rations.loc[:, '12':'17'].sum(axis=1).T/total).T
    df_hour_rations.loc[:, 'evening'] = (df_hour_rations.loc[:, '18':'23'].sum(axis=1).T/total).T
    df_hour_rations.loc[:, 'night'] = (df_hour_rations.loc[:, '00':'04'].sum(axis=1).T/total).T
    
    
# Сводная таблица №3 "количество операций каждого клиента по каждому типу транзацкий" 
    df_tr_type_counts = \
        df_transactions_merged.pivot_table(index=['customer_id'], columns='tr_num', values='amount', 
                          aggfunc=np.size, fill_value=0) 
    
    # переименование просто столбцов с номерами по заданному форомату: "mcc_{}_count"    
    df_tr_type_counts = df_tr_type_counts.rename_axis(lambda x: 'tr_cat_num_{}_count'.format(x), axis=1)
    


    # Объединяем:
    df_features = df_gender.join(df_mcc_counts, on='customer_id', how='left')\
                           .join(df_hour_rations.loc[:, ['morning', 'day', 'evening', 'night']], on='customer_id', how='left')\
                           .join(df_tr_type_counts, on='customer_id', how='left')\
                           #.join(df_transactions_group, on='customer_id', how='left') # категориальный признак
        
        
    return df_features

In [20]:
df_features = df_gender.pipe(gen_features, df_transactions, df_transactions_merged)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead


In [21]:
pd.set_option('display.max_rows', 500)

In [22]:
# объединим еще с категориальными признаками (т.к. в функции что-то не срослось..)
df_features = df_features.merge(df_transactions_group, on='customer_id', how = 'left')

In [23]:
df_features.head().T

,0,1,2,3,4
customer_id,75562265,10928546,69348468,84816985,61009479
gender,0,1,1,1,0
mcc_742_count,0,0,0,0,0
mcc_1711_count,0,0,0,0,0
mcc_1731_count,0,0,0,0,0
mcc_1799_count,0,0,0,0,0
mcc_2741_count,0,0,0,0,0
mcc_3000_count,0,0,0,0,0
mcc_3351_count,0,0,0,0,0
mcc_3501_count,0,0,0,0,0


In [24]:
label = 'gender'
idx_features = df_features.columns != label

X = df_features.loc[:, idx_features].drop('customer_id', axis = 1) # дропнем id
y = df_features.loc[:, ~idx_features] # .values.flatten()  - вдруг пригодится для перевода nd.array

X['total_cat'] = X['total_cat'].apply(pd.to_numeric, errors='coerce') # преобразуем в число
# X = X.values - для перевода nd.array


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Подбор гипер-параметров

In [30]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression

### Pipeline

Сделаем простой sklearn пайплайн, который делает следующее:
* Нормирует признаки через StandartScaler
* Запускает лог-регрессию

### HyperOpt

Есть еще другой вариант - "умный" перебор параметров. И вот тут нам помогает библиотека `hyperopt`.

Если невдаваться в детали, то `hyperopt` определяет следующего набора-кандидата с помощью некоторого алгоритма, который балансирует исследование еще не изведанных областей значений гиперпараметров и направления, вдоль которых наблюдались улучшения целеой метрики.

То есть мы задаем некоторую функцию, и ставим себе цель **минимизировать** (такова договоренность в `hyperopt`) ее значение исходя из параметров, которые она принимает.

In [ ]:
# sudo pip install networkx==1.11

In [31]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, rand

Возвращаясь к нашим баранам, функция вданном случае будет возвращать метрику качества модели (пайплайна) на кроссвалидации. С помощью `hyperopt` мы будем искать минимум этой функции при заданных диаполознах значений гипер параметров.

Но самым важным плюсом, как по мне, является гибкость - в функцию, которую мы хотим минимизировать можно написать все что угодно. Например, сейчас я добавлю вариативность в пайплайне по типу шкалирования между - StandartScaler или RobustScaler.

In [32]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [33]:
def run_trials_template(X, y, params, evals=100):

    def hyperopt_cv(X, y, params):
        
        X_ = X.copy()
        
        # Отделяем параметры лог регрессии в отдельный словарь
        lm_params = {}
        for k, v in params.items(): # было params.iteritems()
            if k.startswith('glob'):
                continue                
            elif k.startswith('lm'):
                lm_params[k.split('_', 1)[1]] = v
        
        # Задаем шкалирование
        if params['scaler_type'] == 'standart':
            scaler = StandardScaler(with_mean=params['scaler_centering'])
        else:
            assert params['scaler_type'] == 'robust'
            scaler = RobustScaler(with_centering=params['scaler_centering'])
        
        # Создаем лог рег с нужными параметрами
        clf = LogisticRegression(**lm_params)
        
        # Итоговый пайплайн
        model = Pipeline([
            ('scaler', scaler),
            ('clf', clf)
        ])

        # Схема кросс-валидации
        n_splits = 5
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, 
                             random_state=RND_SEED)
        scores = cross_val_score(model, X_, y,
                                 scoring='roc_auc', 
                                 cv=cv, 
                                 n_jobs=-1)

        # Возвращаем среднее значение метрики и отклонение (на всякий случай)
        return scores.mean(), scores.std()

    def f(params):
        acc, std = hyperopt_cv(X, y, params)
        return {'loss': -acc, 'qscore': -acc, 'qscore_std': std, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(f, 
                params, 
                algo=tpe.suggest, 
                max_evals=evals, 
                trials=trials, 
                verbose=1)
    
    return trials

In [34]:
# Задаим диапазоны поиска
penalty = ['l1', 'l2']
C_low = -5
C_high = 3
class_weight = [None, 'balanced']
RND_SEED = 123
scaler_type = ['standart', 'robust']
scaler_centering = [False, True]

# Задаем пространство поиска
space4_lm = {
    'lm_penalty': hp.choice('penalty', penalty),
    'lm_C': hp.loguniform('C', C_low, C_high),
    'lm_class_weight': hp.choice('class_weight', class_weight),
    'lm_random_state': RND_SEED,
    'scaler_type': hp.choice('scaler_type', scaler_type),
    'scaler_centering': hp.choice('scaler_centering', scaler_centering)
}

In [35]:
# Запускаем поиск
trials = run_trials_template(X_train, y_train, space4_lm, evals=40)

trials.trials - a list of dictionaries representing everything about the search

trials.results - a list of dictionaries returned by 'objective' during the search

trials.losses() - a list of losses (float for each 'ok' trial)

trials.statuses() - a list of status stringsm

In [36]:
def trials_df(trials):
    '''
    Функция форматирует результаты hyperopt в dataframe
    '''
    tr_dict = []
    for t in trials:
        trial = dict()
        for k, v in t['misc']['vals'].items(): # было iteritems()
            trial[k] = v[0]

        trial['qscore'] = -t['result']['qscore']
        trial['qscore_std'] = -t['result']['qscore_std']
        tr_dict.append(trial)

    df_res = pd.DataFrame.from_dict(tr_dict)
    df_res = df_res.sort_values('qscore', ascending=False)
    
    return df_res

Достаем результаты.

Стоит оговорится, что в некоторых столбцах указаны не фактические значение гиперпараметров, а их позиция в соответствуюем поле в `space4_lm`

In [37]:
df_trials = trials_df(trials)

Здесь `qscore` - метрика качесва, а `scaler_type = 1` означает, что был выбран `scaler_type = robust`

In [38]:
df_trials.head()

,C,class_weight,penalty,qscore,qscore_std,scaler_centering,scaler_type
35,0.202163,0,0,0.829901,-0.009538,0,1
38,0.391246,0,0,0.829224,-0.009446,0,1
26,0.006845,0,1,0.829164,-0.009756,0,1
21,0.006889,0,1,0.829163,-0.009742,0,1
20,0.007079,0,1,0.829154,-0.009747,0,1


In [39]:
best_lm_pre = trials.best_trial['misc']['vals'] # методом best_trial и указав ключи получим словарь с лучшими значениями
best_lm_pre # можно сравнить с датафреймом - да это оно.

{'C': [0.2021634788343108],
 'class_weight': [0],
 'penalty': [0],
 'scaler_centering': [0],
 'scaler_type': [1]}

In [40]:
# Создадим словарь лучших параметров, его потом вставим в логистическую регрессию
best_clf_params = {'C':best_lm_pre['C'][0], \
           'class_weight'     : class_weight[best_lm_pre['class_weight'][0]],
           'penalty'          : penalty[best_lm_pre['penalty'][0]],
          }

# Создадим словарь лучших параметров, его потом вставим в логистическую регрессию
best_clf_params

{'C': 0.2021634788343108, 'class_weight': None, 'penalty': 'l1'}

In [41]:
# Создадим словарь лучших параметров масштабирования

best_scaler_params = {\
           'scaler_type'      : scaler_type[best_lm_pre['scaler_type'][0]],
           'scaler_centering' : scaler_centering[best_lm_pre['scaler_centering'][0]]
          }

# Создадим словарь лучших параметров, потом просто соберем пайплан, глядя на них. 
# Можно в принципе написать функцию **kwargs для выбора скалера, прнимающую на вход этот словарь
best_scaler_params

{'scaler_type': 'robust', 'scaler_centering': False}

### Новые признаки

Задание творческое - придумайте по новому признаку (группе признаков)
* На основе mcc (tr_type)
* На основе временного фактора
* На основе текстов из описания mcc

Реалиуйте их в функции, аналогичной `gen_features`.

## FeatureUnion

In [42]:
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin

Реализуйте функцию для hyperopt по перебору гипер параметров вашего пайплайна

На всякий случай почитайте еще про [`FeatureUnion`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html) и [пример](https://scikit-learn.org/0.19/auto_examples/hetero_feature_union.html)

In [43]:
# создадим списки числовых и категориальных столбцов
full_columns_list = list(X) # полцчим список столбцов - 204
cat_columns = full_columns_list[-1] # список категориальных столбцов
num_columns = full_columns_list[:-1] # список числовых столбцов
#num_columns.remove('gender')

In [44]:
print(num_columns)
print(cat_columns)

['mcc_742_count', 'mcc_1711_count', 'mcc_1731_count', 'mcc_1799_count', 'mcc_2741_count', 'mcc_3000_count', 'mcc_3351_count', 'mcc_3501_count', 'mcc_4111_count', 'mcc_4112_count', 'mcc_4121_count', 'mcc_4131_count', 'mcc_4214_count', 'mcc_4215_count', 'mcc_4411_count', 'mcc_4511_count', 'mcc_4722_count', 'mcc_4784_count', 'mcc_4789_count', 'mcc_4812_count', 'mcc_4814_count', 'mcc_4816_count', 'mcc_4829_count', 'mcc_4899_count', 'mcc_4900_count', 'mcc_5013_count', 'mcc_5039_count', 'mcc_5044_count', 'mcc_5045_count', 'mcc_5047_count', 'mcc_5065_count', 'mcc_5072_count', 'mcc_5074_count', 'mcc_5085_count', 'mcc_5094_count', 'mcc_5099_count', 'mcc_5111_count', 'mcc_5122_count', 'mcc_5131_count', 'mcc_5137_count', 'mcc_5169_count', 'mcc_5172_count', 'mcc_5192_count', 'mcc_5193_count', 'mcc_5199_count', 'mcc_5200_count', 'mcc_5211_count', 'mcc_5231_count', 'mcc_5251_count', 'mcc_5261_count', 'mcc_5300_count', 'mcc_5309_count', 'mcc_5310_count', 'mcc_5311_count', 'mcc_5331_count', 'mcc_5399_

In [45]:
type(cat_columns)

str

In [46]:
# А это трансформер, который выбирает подможнество столбцов из матрицы X
# Который нужен для того, чтобы делать какие-то действия только для подмноества столбцов, а потом объединять результаты
# Через FeatureUnion

# пример из книги. возвращает np.array
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X_, y=None): # 
        return self
    def transform(self, X_):
        return X_.loc[:, self.attribute_names].values # 

In [47]:
num_pipeline = Pipeline([
('selector', ColumnSelector(num_columns)), #  
#('imputer', Imputer(strategy="median")),
('scaler', RobustScaler(with_centering=True)), # в соответствии с best_scaler_params
])

In [48]:
cat_pipeline = Pipeline([
('selector', ColumnSelector([cat_columns])), #именно  [cat_columns] !!!!, иначе будет не матрица, а вектор. НУЖНА МАТРИЦА!
('OHE', OneHotEncoder())
])

In [49]:
featureunion_pipeline = FeatureUnion(transformer_list=[
("pipeline_n1", num_pipeline),
("pipeline_n2", cat_pipeline),
])



### Соберем полные пайплайны

In [50]:
# ПОЛНЫЙ пайплайн С категориальными признаками БЕЗ лучших значений
full_pipeline = Pipeline([
('ftu', featureunion_pipeline),
('clf', LogisticRegression()),
])

In [51]:
# ПОЛНЫЙ пайплайн без категориальных признаков
full_pipeline_2 = Pipeline([
('ftu', num_pipeline),
('clf', LogisticRegression()),
])

In [52]:
# ПОЛНЫЙ пайплайн С категориальными признаками и найденными лучшими значениями
full_pipeline_3 = Pipeline([
('ftu', featureunion_pipeline),
('clf', LogisticRegression(
    **best_clf_params,
    #penalty='l1',  C=0.02539, class_weight='balanced', 
    #penalty='l1',   C=0.03593  # такие grid_searcher.best_params_
)),
])

In [70]:
# Пайплайн для Gridesearcher
num_pipeline_for_gridsearcher = Pipeline([ # 
('selector', ColumnSelector(num_columns)), #  
#('imputer', Imputer(strategy="median")),
('scaler', StandardScaler()), # в соответствии с best_scaler_params
])

featureunion_pipeline_for_gridsearcher = FeatureUnion(transformer_list=[
("pipeline_n1", num_pipeline_for_gridsearcher),
("pipeline_n2", cat_pipeline),
])

full_pipeline_for_gridsearcher = Pipeline([
('ftu', featureunion_pipeline_for_gridsearcher),
('clf', LogisticRegression()),
])

In [53]:
#y = y.values.ravel() # если датафрейм то переведем в nd.array, чтолб питон-зараза не ругался

### Обучим модели

In [54]:
model_2 = full_pipeline_2.fit(X_train,y_train)  # 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
ftu_clf_model = full_pipeline.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for e

In [56]:
ftu_clf_bestparams_model = full_pipeline_3.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for e

### Сделаем прогнозы

In [57]:
predict_1 = ftu_clf_model.predict_proba(X_test)

In [58]:
predict_2 = model_2.predict_proba(X_test)

In [59]:
predict_3 = ftu_clf_bestparams_model.predict_proba(X_test)

### Посмотрим на значения

#### Пайплайн без категориальных признаков

In [61]:
from sklearn.metrics import roc_auc_score

In [62]:
print(roc_auc_score(y_test,predict_2[:,1])) # БЕЗ категориальных признаков

0.8360461323392356


#### Полный пайплайн с категориальными признаками БЕЗ лучших параметров

In [63]:
print(roc_auc_score(y_test,predict_1[:,1]))

0.8432673190431811


#### Пайплайн С категориальными признаками и лучшими параметрами

In [64]:
print(roc_auc_score(y_test,predict_3[:,1]))

0.847736615926271


### Посмотрим что получится на том же пайплайне, но на подобранных через Gridsearch параметрах

In [65]:
from sklearn.model_selection import GridSearchCV

In [84]:
# посмотрим на доступные параметры для param_grid
full_pipeline_for_gridsearcher.get_params().keys()

dict_keys(['memory', 'steps', 'ftu', 'clf', 'ftu__n_jobs', 'ftu__transformer_list', 'ftu__transformer_weights', 'ftu__pipeline_n1', 'ftu__pipeline_n2', 'ftu__pipeline_n1__memory', 'ftu__pipeline_n1__steps', 'ftu__pipeline_n1__selector', 'ftu__pipeline_n1__scaler', 'ftu__pipeline_n1__selector__attribute_names', 'ftu__pipeline_n1__scaler__copy', 'ftu__pipeline_n1__scaler__with_mean', 'ftu__pipeline_n1__scaler__with_std', 'ftu__pipeline_n2__memory', 'ftu__pipeline_n2__steps', 'ftu__pipeline_n2__selector', 'ftu__pipeline_n2__OHE', 'ftu__pipeline_n2__selector__attribute_names', 'ftu__pipeline_n2__OHE__categorical_features', 'ftu__pipeline_n2__OHE__categories', 'ftu__pipeline_n2__OHE__dtype', 'ftu__pipeline_n2__OHE__handle_unknown', 'ftu__pipeline_n2__OHE__n_values', 'ftu__pipeline_n2__OHE__sparse', 'clf__C', 'clf__class_weight', 'clf__dual', 'clf__fit_intercept', 'clf__intercept_scaling', 'clf__max_iter', 'clf__multi_class', 'clf__n_jobs', 'clf__penalty', 'clf__random_state', 'clf__solver',

In [127]:
# Другая версия param_grid, в ней закомтен scaler__with_mean, т.к. в .get_params().keys() его (???) нет
param_grid = {
    'ftu__pipeline_n1__scaler__with_mean': [False, True],
    'clf__penalty': ['l1', 'l2'],
    'clf__random_state': [RND_SEED],
    'clf__C': np.logspace(-5, 3, 10)
}

print(param_grid)

# Задаем схему кросс-валидации
cv = StratifiedKFold(n_splits=5, random_state=RND_SEED, shuffle=True)

{'ftu__pipeline_n1__scaler__with_mean': [False, True], 'clf__penalty': ['l1', 'l2'], 'clf__random_state': [123], 'clf__C': array([1.00000000e-05, 7.74263683e-05, 5.99484250e-04, 4.64158883e-03,
       3.59381366e-02, 2.78255940e-01, 2.15443469e+00, 1.66810054e+01,
       1.29154967e+02, 1.00000000e+03])}


In [128]:
grid_searcher = GridSearchCV(full_pipeline_for_gridsearcher, # пайплайн с feature_union, с категориальными признаками и без лучших значений
                             param_grid, 
                             scoring='roc_auc', 
                             n_jobs=-1, cv=cv, 
                             verbose=2)

grid_searcher.fit(X, y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  6.8min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('ftu', FeatureUnion(n_jobs=None,
       transformer_list=[('pipeline_n1', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(attribute_names=['mcc_742_count', 'mcc_1711_count', 'mcc_1731_count', 'mcc_1799_count', 'mcc_2741_count', 'mcc_3000_count', 'mcc_3351_count', 'mcc_3501_coun...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'ftu__pipeline_n1__scaler__with_mean': [False, True], 'clf__penalty': ['l1', 'l2'], 'clf__random_state': [123], 'clf__C': array([1.00000e-05, 7.74264e-05, 5.99484e-04, 4.64159e-03, 3.59381e-02,
       2.78256e-01, 2.15443e+00, 1.66810e+01, 1.29155e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', v

In [129]:
grid_searcher.best_params_

{'clf__C': 0.03593813663804626,
 'clf__penalty': 'l1',
 'clf__random_state': 123,
 'ftu__pipeline_n1__scaler__with_mean': True}

In [130]:
grid_searcher.best_score_

0.8398784418943074

In [131]:
grid_best_model = grid_searcher.best_estimator_

In [132]:
predict_grid_best_model = grid_best_model.predict_proba(X_test)

In [133]:
print(roc_auc_score(y_test,predict_grid_best_model[:,1]))

0.861819405612509


Итоги:

* модель **ftu_clf_bestparams_model** С  доп.обработкой фитч.(на данном ноутбуке) дала       roc_auc_score: **0.85045**   
* модель **predict_grid_best_model** без кат.признаков С  доп.обработкой фитч.(на данном ноутбуке) дала       roc_auc_score: **0.8627**
* модель **predict_grid_best_model** C кат.признаками С  доп.обработкой фитч.(на данном ноутбуке) дала       roc_auc_score: **0.8618**
* **Полный** пайплайн с категориальными признаками но все еще БЕЗ лучших параметров          roc_auc_score: **0.8432**
* Пайплайн БЕЗ категориальных признаков и БЕЗ лучших параметров                            roc_auc_score: **0.83604**
* модель на **best_grid_search**     без доп.обработки фитч. (на лекционном ноутбуке) давала roc_auc_score: **0.8333**

### А еще можно собрать пайплайн через ColumnTransformer.  Получается один в один

In [92]:
 from sklearn.compose import ColumnTransformer

In [144]:
ct = ColumnTransformer([
    ('OHE', OneHotEncoder(), [204]), # 
    ('scaler',StandardScaler(), slice(0,204))
     ])

full_pipeline_4 = Pipeline([
('ct', ct),
('clf', LogisticRegression())
])

In [148]:
full_pipeline_4.get_params().keys()

dict_keys(['memory', 'steps', 'ct', 'clf', 'ct__n_jobs', 'ct__remainder', 'ct__sparse_threshold', 'ct__transformer_weights', 'ct__transformers', 'ct__OHE', 'ct__scaler', 'ct__OHE__categorical_features', 'ct__OHE__categories', 'ct__OHE__dtype', 'ct__OHE__handle_unknown', 'ct__OHE__n_values', 'ct__OHE__sparse', 'ct__scaler__copy', 'ct__scaler__with_mean', 'ct__scaler__with_std', 'clf__C', 'clf__class_weight', 'clf__dual', 'clf__fit_intercept', 'clf__intercept_scaling', 'clf__max_iter', 'clf__multi_class', 'clf__n_jobs', 'clf__penalty', 'clf__random_state', 'clf__solver', 'clf__tol', 'clf__verbose', 'clf__warm_start'])

In [149]:
# Другая версия param_grid, в ней закомтен scaler__with_mean, т.к. в .get_params().keys() его (???) нет
param_grid_2 = {
    'ct__scaler__with_mean': [False, True],
    'clf__penalty': ['l1', 'l2'],
    'clf__random_state': [RND_SEED],
    'clf__C': np.logspace(-5, 3, 10)
}

print(param_grid)

# Задаем схему кросс-валидации
cv = StratifiedKFold(n_splits=5, random_state=RND_SEED, shuffle=True)

{'ftu__pipeline_n1__scaler__with_mean': [False, True], 'clf__penalty': ['l1', 'l2'], 'clf__random_state': [123], 'clf__C': array([1.00000000e-05, 7.74263683e-05, 5.99484250e-04, 4.64158883e-03,
       3.59381366e-02, 2.78255940e-01, 2.15443469e+00, 1.66810054e+01,
       1.29154967e+02, 1.00000000e+03])}


In [150]:
grid_searcher_ct = GridSearchCV(full_pipeline_4, # пайплайн с ColumnTransformer, с категориальными признаками и без лучших значений
                             param_grid_2, 
                             scoring='roc_auc', 
                             n_jobs=-1, cv=cv, 
                             verbose=2)

grid_searcher_ct.fit(X, y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  8.6min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were al

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('ct', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('OHE', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'ct__scaler__with_mean': [False, True], 'clf__penalty': ['l1', 'l2'], 'clf__random_state': [123], 'clf__C': array([1.00000e-05, 7.74264e-05, 5.99484e-04, 4.64159e-03, 3.59381e-02,
       2.78256e-01, 2.15443e+00, 1.66810e+01, 1.29155e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [151]:
grid_searcher_ct.best_params_

{'clf__C': 0.03593813663804626,
 'clf__penalty': 'l1',
 'clf__random_state': 123,
 'ct__scaler__with_mean': True}

In [152]:
grid_best_model_ct = grid_searcher_ct.best_estimator_

In [153]:
predict_grid_best_model_ct = grid_best_model_ct.predict_proba(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:605: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


In [154]:
print(roc_auc_score(y_test,predict_grid_best_model_ct[:,1]))

0.861819405612509
